# 데이터 불러오기

In [2]:
import pandas as pd
import os

In [3]:

iris = pd.read_csv("/content/drive/My Drive/Colab Notebooks/data/IRIS.csv")

In [5]:
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [6]:
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [7]:
iris['species'].value_counts()

Iris-virginica     50
Iris-versicolor    50
Iris-setosa        50
Name: species, dtype: int64

In [8]:

iris['id'] = range(len(iris))

In [9]:
## 열의 순서를 바꿉니다.
iris = iris[['id','sepal_length','sepal_width','petal_length','petal_width','species']]

# knn 실습(분류)

<strong>특징</strong>
1. 데이터가 많으면 느리다.

<strong>파라미터</strong>
1. n_neighbors: 가장 가까운 몇 개를 볼건지

<strong>knn 학습시키기</strong>

In [10]:
## iris 중에서 100개를 랜덤으로 추출해 train 데이터로 지정
train = iris.sample(100,replace=False,random_state=10).reset_index().drop(['index'],axis=1)

In [11]:
## 추출되지 않은 나머지를 test 데이터로 지정합니다.
test = iris.loc[ ~iris['id'].isin(train['id']) ]
test = test.reset_index().drop(['index'],axis=1)

In [12]:
## knn을 위한 패키지를 임포트합니다.
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=2)

In [16]:
train_x = train[['sepal_length','sepal_width','petal_length','petal_width']]
train_y = train['species']
test_x = test[['sepal_length','sepal_width','petal_length','petal_width']]
test_y = test['species']

In [15]:
## knn 모델을 train 데이터에서 학습시킵니다.
knn.fit( train_x,train_y )

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [17]:
## 예측을 진행합니다.
predictions = knn.predict(test_x)

In [18]:
## 정확도를 살펴봅니다.
(pd.Series(predictions) == test_y).mean()

0.92

<strong>최적의 k 찾기</strong>

In [20]:

for k in range(1,30):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit( train_x , train_y )
    predictions = knn.predict( test_x )
    print((pd.Series(predictions) == test_y).mean())

0.96
0.92
0.94
0.92
0.92
0.92
0.94
0.92
0.94
0.92
0.94
0.92
0.94
0.94
0.94
0.9
0.9
0.88
0.92
0.88
0.92
0.92
0.94
0.92
0.94
0.9
0.92
0.9
0.9


## 최적의 k 는 1

In [21]:
from sklearn.model_selection import cross_val_score
import numpy as np

<strong>Cross Validation</strong>

Cross validation을 통해 최적의 k를 찾습니다.<br>


In [22]:
for k in range(1,30):
     knn = KNeighborsClassifier(n_neighbors=k)
     scores = cross_val_score(knn, iris[['sepal_length','sepal_width','petal_length','petal_width']], iris['species'], cv=5)
     print(np.mean(scores))

0.96
0.9466666666666665
0.9666666666666668
0.9733333333333334
0.9733333333333334
0.9800000000000001
0.9800000000000001
0.9666666666666668
0.9733333333333334
0.9800000000000001
0.9800000000000001
0.9800000000000001
0.9733333333333334
0.9666666666666666
0.9666666666666668
0.9666666666666668
0.9666666666666668
0.9666666666666666
0.9666666666666668
0.96
0.9666666666666668
0.96
0.96
0.9466666666666667
0.96
0.9466666666666665
0.9466666666666667
0.9399999999999998
0.9333333333333332


# knn 회귀

In [23]:
## 회귀를 위해 필요한 컬럼을 제거
del train['species']
del test['species']

In [24]:

from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(n_neighbors=2)

In [25]:
train_x = train[['sepal_length','sepal_width','petal_length']]
train_y = train['petal_width']
test_x = test[['sepal_length','sepal_width','petal_length']]
test_y = test['petal_width']

In [26]:
## knn 모델을 학습

knn.fit( train_x , train_y )

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                    weights='uniform')

In [27]:
## 예측합니다.

predictions = knn.predict( test_x )

In [28]:
predictions

array([0.25, 0.2 , 0.1 , 0.3 , 0.25, 0.2 , 0.15, 0.2 , 0.3 , 0.25, 0.3 ,
       0.1 , 0.25, 0.25, 0.15, 0.2 , 0.15, 0.2 , 0.25, 1.35, 1.05, 1.3 ,
       1.1 , 1.35, 1.05, 1.6 , 1.35, 1.9 , 1.4 , 1.25, 1.2 , 1.1 , 1.05,
       1.3 , 2.35, 2.  , 1.9 , 1.9 , 1.7 , 2.2 , 1.65, 1.85, 1.6 , 2.  ,
       2.  , 2.15, 1.4 , 2.15, 1.9 , 2.  ])

<strong>mae</strong>

회귀문제에서 모델의 성능을 평가하는 방법

In [29]:
abs(test_y - pd.Series(predictions)).mean()

0.149

<strong>최적의 k 찾기</strong>

In [30]:
## 최적의 k를 찾습니다.

for k in range(1,30):
    knn = KNeighborsRegressor(n_neighbors=k)

    knn.fit( train_x, train_y )

    predictions = knn.predict( test_x )
    
    print(str(k)+' :'+str(abs(test_y - pd.Series(predictions)).mean()))

1 :0.17399999999999996
2 :0.149
3 :0.14933333333333332
4 :0.145
5 :0.14239999999999994
6 :0.14066666666666663
7 :0.14085714285714285
8 :0.13874999999999998
9 :0.13844444444444437
10 :0.14120000000000002
11 :0.1438181818181818
12 :0.1478333333333333
13 :0.14692307692307685
14 :0.14685714285714283
15 :0.14626666666666668
16 :0.14875000000000002
17 :0.15176470588235294
18 :0.15088888888888882
19 :0.15410526315789475
20 :0.15869999999999995
21 :0.15904761904761905
22 :0.16209090909090906
23 :0.16591304347826083
24 :0.16833333333333328
25 :0.16840000000000002
26 :0.1706153846153847
27 :0.17288888888888887
28 :0.1761428571428571
29 :0.1789655172413793


최적의 k는 9